In [2]:
%matplotlib widget
import warnings
import fnmatch
import os
#import ccdproc
from ccdproc import Combiner, combine, subtract_bias, subtract_dark
from astropy.utils.exceptions import AstropyWarning
from time import gmtime, strftime
import numpy as np
import matplotlib.pyplot as plt

from astropy.table import Table
from astropy import units as u

from astropy.nddata import CCDData, StdDevUncertainty
from specutils import Spectrum1D
import ccdproc
from astropy.stats import mad_std
from ccdproc import trim_image, Combiner
from astropy.io import fits
from time import gmtime, strftime

#warnings.simplefilter('ignore', category=AstropyWarning)

dir = 'e:/Astro/Captures/20231008_Void/'
bias_files = [dir + f for f in fnmatch.filter(os.listdir(dir), 'Bias*')]
dark_files = [dir + f for f in fnmatch.filter(os.listdir(dir), 'dark*')]
flat_files = [dir + f for f in fnmatch.filter(os.listdir(dir), 'flat*')]
sci_files  = [dir + f for f in fnmatch.filter(os.listdir(dir), 'deneb*')]

bias_header = fits.open(bias_files[0])[0].header
dark_header = fits.open(dark_files[0])[0].header
flat_header = fits.open(flat_files[0])[0].header
sci_header = fits.open(sci_files[0])[0].header

print ('bias combine...')
from datetime import datetime; print(datetime.now())
bias_list = [CCDData.read(f, unit = u.adu, dtype=np.float32) for f in bias_files]
bias_master = ccdproc.combine(bias_list, method='median', dtype=np.float32, mem_limit = 1e9)
bias_list = None
out_file = fits.PrimaryHDU(bias_master.data, bias_header)
out_hdr = out_file.header
out_hdr['DATE'] = strftime("%Y-%m-%dT%H:%M:%S", gmtime())
out_hdr['history'] = 'numpy.median combined'
out_file.writeto(dir + 'masterbias.fit', overwrite = True)

from datetime import datetime; print(datetime.now())
darks_list = []
for dark_file in dark_files:
    dark_data = CCDData.read(dark_file, unit = u.adu, dtype=np.float32)
    print('dark_data', str(dark_data.dtype))
    dark_sub = ccdproc.subtract_bias(dark_data, bias_master)
    dark_sub.data = dark_sub.data.astype('float32')
    dark_data = None
    print('dark_sub', str(dark_sub.dtype))
    darks_list.append(dark_sub)

print ('dark combine...')
dark_master = ccdproc.combine(darks_list, method='median', dtype=np.float32, mem_limit = 1e9)
darks_list = None

#dark_master = ccdproc.subtract_bias(dark_datas, bias_master, dtype = np.float32, mem_limit = 250e6)
out_file = fits.PrimaryHDU(dark_master, dark_header)
out_hdr = out_file.header
out_hdr['DATE'] = strftime("%Y-%m-%dT%H:%M:%S", gmtime())
out_hdr['history'] = 'numpy.median combined'
out_file.writeto(dir + 'masterdark.fit', overwrite = True)
dark_datas = None

"""
print ('flat combine...')
from datetime import datetime; print(datetime.now())
flat_master = np.median([(fits.open(f)[0].data.astype(np.uint16) - bias_master) for f in flat_files], axis=0)

out_file = fits.PrimaryHDU(flat_master, flat_header)
out_hdr = out_file.header
out_hdr['DATE'] = strftime("%Y-%m-%dT%H:%M:%S", gmtime())
out_hdr['history'] = 'numpy.median combined'
out_file.writeto(dir + 'masterflat.fit', overwrite = True)

print ('sci combine...')
from datetime import datetime; print(datetime.now())
sci_master = np.sum([((fits.open(f)[0].data.astype(np.uint16) - bias_master - dark_master) / 1) for f in sci_files], axis=0)

out_file = fits.PrimaryHDU(sci_master.astype('float32'), sci_header)
out_hdr = out_file.header
out_hdr['DATE'] = strftime("%Y-%m-%dT%H:%M:%S", gmtime())
out_hdr['history'] = 'numpy.sum combined'
out_file.writeto(dir + 'mastersci.fit', overwrite = True)
"""
"""
darkhdr = darklist[0].header 
inexp = float(inhdr['EXPTIME'])
darkexp = float(darkhdr['EXPTIME'])
darkscale = inexp/darkexp
outimage = (inimage -inbias) - darkscale * (indark -inbias)
"""

bias_master = None
dark_master = None
flat_master = None
sci_master = None
bias_header = None
flat_header = None
dark_header = None
sci_header = None


bias combine...
2024-02-12 20:44:25.511609
INFO: splitting each image into 11 chunks to limit memory usage to 1000000000.0 bytes. [ccdproc.combiner]
2024-02-12 20:45:55.822918
dark_data uint16
dark_sub float32
dark_data uint16
dark_sub float32
dark_data uint16
dark_sub float32
dark_data uint16
dark_sub float32
dark_data uint16
dark_sub float32
dark_data uint16
dark_sub float32
dark_data uint16
dark_sub float32
dark_data uint16
dark_sub float32
dark_data uint16
dark_sub float32
dark_data uint16
dark_sub float32
dark_data uint16
dark_sub float32
dark_data uint16
dark_sub float32
dark_data uint16
dark_sub float32
dark_data uint16
dark_sub float32
dark_data uint16
dark_sub float32
dark combine...
INFO: splitting each image into 11 chunks to limit memory usage to 1000000000.0 bytes. [ccdproc.combiner]


In [2]:
from astropy.io import fits
from time import gmtime, strftime

def fits_combine(images_list: list[str], operation, outfile: str) -> np:
    images = [fits.open(f)[0].data for f in images_list]
    headers = [fits.open(f)[0].header for f in images_list]
    new_image = operation(images, axis=0)
    out_file = fits.PrimaryHDU(new_image.astype(np.float32), headers[0])
    out_hdr = out_file.header
    out_hdr['DATE'] = strftime("%Y-%m-%dT%H:%M:%S", gmtime())
    out_hdr['history'] = '{} of {} images'.format(str(operation), len(images_list))
    out_file.writeto(outfile, overwrite = True)
    images = None
    return (new_image)

def combine_fits(files_data: list[np], operation, outfile: str) -> None:
    img_array = []
    hdu_array = []
    nb_images = 0
   
    median_array = operation(img_array, axis=0)
    out_file = fits.PrimaryHDU(median_array, hdu_array[0])
    out_hdr = out_file.header
    out_hdr['DATE'] = strftime("%Y-%m-%d %H:%M:%S", gmtime())
    out_hdr['history'] = 'Median of %d images by numpy.median' %(nb_images,)
    out_hdr['history'] = 'First image '+  files[0]
    out_hdr['history'] = 'Last image  '+  files[nb_images - 1]
    out_file.writeto(outfile, overwrite = True)


In [2]:
bias_master = None
dark_mater = None
flat_master = None
sci_master = None
dark_header = None
sci_header = None


In [ ]:
"""
inhdr = inlist[0].header
darkhdr = darklist[0].header 
inexp = float(inhdr['EXPTIME'])
darkexp = float(darkhdr['EXPTIME'])
darkscale = inexp/darkexp


# Create a dark-subtracted image

outimage = (inimage -inbias) - darkscale * (indark -inbias)

combine_fits(bias_files, np.median, dir + 'masterbias.fit')

darks_list = []
masterbias = CCDData.read(dir + 'masterbias.fit', unit = u.adu)
for dark in darkfiles:
    dark = CCDData.read(dark, unit = u.adu)
    dark_sub = ccdp.subtract_bias(dark, masterbias)
    darks_list.append(dark_sub)

dark_sub_file = ccdproc.combine(blue_flat_list, method='median')
dark_sub_file.write('master_flat_blue.fits', clobber=True)



combine_fits(biasfiles, np.median, dir + 'masterdark.fit')
"""

In [ ]:
plt.figure(figsize=(10,6))
plt.imshow(outimage, origin='lower', aspect='auto', cmap=plt.cm.inferno)
plt.title('outimage')
plt.clim(np.percentile(outimage, (5, 98)))
cb = plt.colorbar()

In [ ]:
import fnmatch
import os
dir = 'e:/Astro/Captures/20231008_Void/'
biasfiles = [dir + f for f in fnmatch.filter(os.listdir(dir), 'Bias*')]
combine_median(biasfiles, dir + 'masterbias.fit')
scifile = 'deneb-200s-1.fit'
masterbias = CCDData.read(dir + 'masterbias.fit', unit = u.adu)
ccd_bias = ccdp.subtract_bias(ccd, masterbias)

#masterdark = CCDData.read(dir + 'masterdark.fit', unit = u.adu)
#masterflat = CCDData.read(dir + 'masterflat.fit', unit = u.adu)
ccd_dark = ccdp.subtract_dark(ccd_bias, masterdark, exposure_unit=u.second, scale = True, dark_exposure = 300.0*u.second, data_exposure = 200.0*u.second)
ccd = ccdp.flat_correct(ccd_dark, masterflat, min_value=0.9)
ccd = CCDData.read(dir + scifile, unit = u.adu)
plt.figure(figsize=(10,6))
plt.imshow(ccd, origin='lower', aspect='auto', cmap=plt.cm.gray)
plt.title('outimage')
#plt.clim(np.percentile(ccd, (5, 98)))
cb = plt.colorbar()

In [ ]:
from specreduce import tracing, background, extract
sci_tr = tracing.FitTrace(ccd,  peak_method='max')   #FitTrace(image, peak_method='gaussian', guess=trace_pos)
bg = background.Background.two_sided(ccd, sci_tr, separation=50, width=20) 
extract = extract.BoxcarExtract(ccd - bg, sci_tr, width = 10)
spectrum = extract()


In [ ]:
extract.width


In [ ]:
plt.figure(figsize=(10,6))
plt.imshow(ccd, origin='lower', aspect='auto', cmap=plt.cm.gray)
plt.imshow(bg.bkg_wimage, origin='lower', aspect='auto', cmap=plt.cm.gray, alpha=0.5)
plt.plot(sci_tr.trace , color='r')
plt.plot(sci_tr.trace+extract.width , color='g', linestyle='dashed')
plt.plot(sci_tr.trace-extract.width , color='g', linestyle='dashed')

cb = plt.colorbar()



In [ ]:
plt.figure(figsize=(10,6))
#plt.imshow(bg.sub_image(ccd).data, origin='lower', aspect='auto', cmap=plt.cm.gray)
#plt.imshow(bg.sub_image(ccd).data, origin='lower', aspect='auto', cmap=plt.cm.gray)
plt.plot(spectrum.flux, color='r')
#cb = plt.colorbar()

